In [392]:
!pip install pandas numpy tensorflow


In [393]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



In [394]:
def predict_next_7_days(model, scaler, last_known_data):
    predictions = []
    input_data = last_known_data.reshape(1, 1, -1)
    for _ in range(7):
        next_pred = model.predict(input_data, verbose=0)
        predictions.append(next_pred[0, 0])

        # Update input_data with the new prediction
        next_input = np.roll(input_data, -1, axis=2)
        next_input[0, 0, -1] = next_pred
        input_data = next_input

    # Create a placeholder with the correct shape for inverse_transform
    predictions_full = np.zeros((len(predictions), scaled_data.shape[1]))

    # Assign predictions to the first column (representing 'Close')
    predictions_full[:, 0] = predictions

    # Inverse transform and return the 'Close' column
    return scaler.inverse_transform(predictions_full)[:, 0]

In [395]:
def predict_for_user_input(ticker_value, date_input):
    # Load data for the specific ticker
    data = pd.read_csv(f'TRAIN/{ticker_value}.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data['Rolling_Avg'] = data['Close'].rolling(window=window_size).mean()
    data.dropna(inplace=True)

    for i in range(1, 16):
      data[f'Lag_{i}'] = data['Close'].shift(i)

  # Drop rows with NaN values caused by lagging
  #data.dropna(inplace=True) # Original line causing the error
  # Instead of dropping all rows with NaN, drop only those where all Lag columns are NaN
    data.dropna(subset=[f'Lag_{i}' for i in range(1, 11)], how='all', inplace=True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[['Close'] + [f'Lag_{i}' for i in range(1, 11)]])


    # Initialize predicted_values to an empty list to avoid UnboundLocalError
    predicted_values = []  # This line is added

    if date_input in data['Date'].values:
        print(f"Date {date_input} exists in the dataset.")
        # ... (rest of your code remains the same) ...

    else:
        print(f"Date {date_input} is not present in the dataset. Predicting future values...")
        #  Get the last 15 values, including Rolling_Avg (for future predictions)
        row_for_prediction = data.iloc[-1, -15:].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(data[['Close'] + [f'Lag_{i}' for i in range(1, 11)]])
        predicted_values = predict_next_7_days(model, scaler, row_for_prediction[1:])

    # Now you can safely use predicted_values in the for loop:
    print("\nNext 7 Days Predictions:")
    for date, predicted in zip(future_dates, predicted_values):
        print(f"{date.date()}: Predicted Close = {predicted:.2f}")

    return scaled_data,scaler

In [396]:
tickers = [f.split('.')[0] for f in os.listdir('TRAIN') if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
all_data = []

# Loop over each ticker and process
for ticker_value in tickers:
    print(f"Processing {ticker_value}...")

    # Load the stock data for the current ticker
    data = pd.read_csv(f'TRAIN/{ticker_value}.csv')

    # Add a column to identify the ticker
    data['Ticker'] = ticker_value

    # Add the rolling average column
    data['Rolling_Avg'] = data['Close'].rolling(window=window_size).mean()
    data.dropna(inplace=True)  # Remove rows with NaN values caused by rolling
    all_data.append(data)
    combined_data = pd.concat(all_data, ignore_index=True)

# Print the combined DataFrame with the rolling average
print(combined_data)

Processing VEDL...
Processing UPL...
Processing ONGC...
Processing ZEEL...
Processing TCS...
Processing ULTRACEMCO...
Processing WIPRO...
Processing TECHM...
Processing TATASTEEL...
Processing TITAN...
             Date   Symbol Series  Prev Close     Open     High      Low  \
0      2011-06-01  SESAGOA     EQ      289.90   289.90   295.45   289.90   
1      2011-06-02  SESAGOA     EQ      293.35   289.65   289.75   282.10   
2      2011-06-03  SESAGOA     EQ      283.05   283.30   291.70   283.10   
3      2011-06-06  SESAGOA     EQ      289.30   288.60   289.85   281.10   
4      2011-06-07  SESAGOA     EQ      285.30   284.00   288.90   282.40   
...           ...      ...    ...         ...      ...      ...      ...   
24555  2021-04-26    TITAN     EQ     1461.45  1470.00  1489.75  1465.15   
24556  2021-04-27    TITAN     EQ     1479.85  1485.00  1503.55  1482.00   
24557  2021-04-28    TITAN     EQ     1495.10  1505.00  1514.90  1491.70   
24558  2021-04-29    TITAN     EQ     

In [397]:
 # Example window size
data['Rolling_Avg'] = data['Close'].rolling(window=window_size).mean()
data.dropna(inplace=True)  # Remove rows with NaN values caused by rolling


In [398]:
scaled_data,scaler=predict_for_user_input('TCS','2005/02/02')

Date 2005/02/02 is not present in the dataset. Predicting future values...


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.float64).

In [ ]:
X = scaled_data[:, 1:]  # All columns except the target (Close)
y = scaled_data[:, 0]   # Target is the 'Close' column

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape for LSTM input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
model = Sequential([
    LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=15, validation_data=(X_test, y_test))


In [ ]:
predictions = model.predict(X_test)

# Placeholder for predictions inverse transformation
predictions_full = np.zeros((predictions.shape[0], scaled_data.shape[1]))
predictions_full[:, 0] = predictions.flatten()
predictions_inverse = scaler.inverse_transform(predictions_full)[:, 0]

# For actuals
actuals_full = np.zeros((y_test.shape[0], scaled_data.shape[1]))
actuals_full[:, 0] = y_test.flatten()
actuals_inverse = scaler.inverse_transform(actuals_full)[:, 0]


# Include Rolling Average for comparison
rolling_avg_actuals = scaler.inverse_transform(actuals_full)[:, 1]
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(actuals_inverse, label='Actual Close Values')
plt.plot(rolling_avg_actuals, label='Rolling Average')
plt.plot(predictions_inverse, label='Predicted Close Values')
plt.legend()
plt.title('Actual vs Rolling Average vs Predicted ('+ticker_value+')')
plt.xlabel('Time Steps')
plt.ylabel('Close Value')
plt.show()



In [ ]:
data.columns


In [ ]:
# Print results
#last_date = pd.to_datetime(data['Date'].iloc[-1])  # Convert last_date to datetime
future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 8)]
print("\nNext 7 Days Predictions:")
for date, value in zip(future_dates, predicted_values):
    print(f"{date.date()}: Predicted Close = {value:.2f}")